In [1]:
from math import log
import numpy as np
import matplotlib.pyplot as plt

from scipy.optimize import fsolve
from scipy.integrate import nquad, quad
from scipy.special import logsumexp
import scipy.stats as st

import pymc as pm
import arviz as az
import seaborn as sb

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
np.random.seed(42)
N = 500
X_vec = np.random.uniform(-10,10,N)
true_beta = 2
true_sigma = 5
epsilon = np.random.randn(N) * true_sigma
y_vec = X_vec * true_beta + epsilon
S_1 = 400
sig2 = 10

In [3]:
# Define the model
with pm.Model() as model:
    # Priors
    beta = pm.Normal('beta', mu=0, sigma=np.sqrt(S_1))
    log_sigma2 = pm.Normal('log_sigma2', mu=0, sigma=(sig2))
    sigma2 = pm.Deterministic('sigma2', pm.math.exp(log_sigma2))
    sigma = pm.Deterministic('sigma', pm.math.sqrt(sigma2))
    
    # Likelihood
    likelihood = pm.Normal('y', mu=X_vec * beta, sigma=sigma, observed=y_vec)
    
    # Sampling
    trace = pm.sample(1000, tune=1000, chains=4, return_inferencedata=True)
    
    # Compute log likelihood
    log_likelihood = pm.compute_log_likelihood(trace)
    
    # Estimating marginal likelihood using logp and LOO-CV
    loo_result = az.loo(trace, pointwise=True)
    print(loo_result)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta, log_sigma2]


/Users/uliraudales/Desktop/Research/IFT/IFT_Code/.venv/lib/python3.11/site-packages/rich/live.py:231: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 1 seconds.


/Users/uliraudales/Desktop/Research/IFT/IFT_Code/.venv/lib/python3.11/site-packages/rich/live.py:231: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Computed from 4000 posterior samples and 500 observations log-likelihood matrix.

         Estimate       SE
elpd_loo -1518.15    15.00
p_loo        2.09        -
------

Pareto k diagnostic values:
                         Count   Pct.
(-Inf, 0.70]   (good)      500  100.0%
   (0.70, 1]   (bad)         0    0.0%
   (1, Inf)   (very bad)    0    0.0%



In [4]:
# Access posterior samples
posterior_samples = trace.posterior

# Variables in the posterior
print(posterior_samples)

<xarray.Dataset> Size: 136kB
Dimensions:     (chain: 4, draw: 1000)
Coordinates:
  * chain       (chain) int64 32B 0 1 2 3
  * draw        (draw) int64 8kB 0 1 2 3 4 5 6 7 ... 993 994 995 996 997 998 999
Data variables:
    beta        (chain, draw) float64 32kB 2.076 2.09 2.0 ... 2.085 2.05 2.1
    log_sigma2  (chain, draw) float64 32kB 3.213 3.253 3.239 ... 3.202 3.214
    sigma       (chain, draw) float64 32kB 4.986 5.087 5.052 ... 4.959 4.988
    sigma2      (chain, draw) float64 32kB 24.86 25.88 25.52 ... 24.59 24.88
Attributes:
    created_at:                 2024-09-20T18:10:50.913466+00:00
    arviz_version:              0.19.0
    inference_library:          pymc
    inference_library_version:  5.16.2
    sampling_time:              1.4769182205200195
    tuning_steps:               1000


In [5]:
L = 1000

# Flatten the chains and draws to get a single dimension
beta_samples = posterior_samples['beta'].values.flatten()
sigma_samples = posterior_samples['sigma'].values.flatten()

# Total available samples
total_samples = beta_samples.shape[0]  # Should be 4000

# Check if L is less than or equal to total_samples
if L > total_samples:
    print(f"Requested {L} samples, but only {total_samples} are available.")
    L = total_samples

# Obtain L posterior draws
beta_L_samples = beta_samples[:L]
sigma_L_samples = sigma_samples[:L]


In [6]:
# Example: Calculate posterior mean estimates
beta_mean = np.mean(beta_L_samples)
sigma_mean = np.mean(sigma_L_samples)

print(f"Posterior mean of beta (based on {L} samples): {beta_mean}")
print(f"Posterior mean of sigma (based on {L} samples): {sigma_mean}")


Posterior mean of beta (based on 1000 samples): 2.0583383434226126
Posterior mean of sigma (based on 1000 samples): 5.031032657947175


In [7]:
# Recalculate log likelihood manually
beta_samples = trace.posterior['beta'].values
sigma_samples = trace.posterior['sigma'].values

# Recalculate the log likelihood for each sample
log_likelihoods = []
for beta, sigma in zip(beta_samples.flatten(), sigma_samples.flatten()):
    logp = np.sum(-0.5 * np.log(2 * np.pi * (sigma ** 2)) - ((y_vec - X_vec * beta) ** 2) / (2 * (sigma ** 2)))
    log_likelihoods.append(logp)

log_likelihoods = np.array(log_likelihoods)

# Calculate the log marginal likelihood using the harmonic mean
log_ml = -logsumexp(-log_likelihoods) + np.log(len(log_likelihoods))

print(f"Estimated log marginal likelihood: {log_ml}")

Estimated log marginal likelihood: -1518.2216971423475
